## Introduction/Business Problem

Problem

Climate events are causing accidents on the roads of Seattle. Accidents can have severe mental and physical impacts on those directly involved and also reduce the efficiency and effectiveness of transport infrastructure. 

We want to understand if it is possible to use data from past accidents with corresponding weather data to predict the probability of accidents and of the severity of these accidents.   

If we find that that there is a relationship between weather and accidents then we can use this data to educate road users and / or to put weather specific safety measures in place. We would thus hope to reduce the direct and indirect costs of accidents.

Conclusions drawn from this data set may also be applicable to other other parts of the country / world. We can assess the model to see if this is the case.


